Declaration of libraries

In [55]:
import pandas as pd
import numpy as np
import pyodbc
pd.set_option("display.max_columns", 999)
pd.set_option("display.max_rows", 999)

SQL query to gather some sample data

In [22]:
driver = '{SQL Server Native Client 11.0}'
server = 'sql2012'
db = 'og_irasv6_stage'
conn = pyodbc.connect("Driver="+driver+";Server="+server+";Database="+db+";Trusted_Connection=yes;")

# performing query to database

q1 = """set nocount on;
        select * from mop"""

q2 = """set nocount on;
        select ld1.code [material_type], ld2.code [seam_type], ld3.code [gwd_type], ld4.code [manufacturer], ps.* from pipesegment ps
        join listdomain ld1 on ps.PipeMaterialDomainId = ld1.id
        join listdomain ld2 on ps.PipeSeamDomainId = ld2.id
        join listdomain ld3 on ps.PipeGirthWeldDomainID = ld3.id
        join listdomain ld4 on ps.PipeManufacturerDomainId = ld4.id"""

mop = pd.read_sql_query(q1,conn)
ps = pd.read_sql_query(q2,conn)
ps.loc[:,'PipeInserviceDate'] =  pd.to_datetime(ps.loc[:,'PipeInserviceDate'])

conn.close()

In [10]:
mop.describe(include='all').loc[:,'DesignPressure']

count      7204.000000
unique             NaN
top                NaN
freq               NaN
first              NaN
last               NaN
mean       5316.992000
std        4141.115604
min           6.894800
25%        3033.693200
50%        5102.120400
75%        5102.120400
max       68940.678100
Name: DesignPressure, dtype: float64

In [23]:
ps.describe(include='all').loc[:,['PipeInserviceDate',
                                  'PipeOutsideDiameter',
                                  'PipeWallThickness',
                                  'PipeGrade',
                                  'PipeToughness',
                                  'material_type',
                                  'seam_type',
                                  'gwd_type',
                                  'manufacturer']]

,PipeInserviceDate,PipeOutsideDiameter,PipeWallThickness,PipeGrade,PipeToughness,material_type,seam_type,gwd_type,manufacturer
count,0.0,1230.000000,1218.000000,1048.000000,0.0,1230,1230,1230,1230
unique,0.0,NaN,NaN,NaN,0.0,1,7,5,11
top,NaN,NaN,NaN,NaN,NaN,Steel,Electric Resistance Weld (ERW),Unknown,Unknown
freq,NaN,NaN,NaN,NaN,NaN,1230,724,1037,1080
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,11.193374,6.102603,308.926527,NaN,NaN,NaN,NaN,NaN
std,NaN,6.826422,1.635748,49.949615,NaN,NaN,NaN,NaN,NaN
min,NaN,1.315000,2.770000,206.000000,NaN,NaN,NaN,NaN,NaN
25%,NaN,4.500000,4.780000,290.000000,NaN,NaN,NaN,NaN,NaN


Loading the data variable list, and filtering out the fields in order to only obtain event tables

In [47]:
dvl = pd.read_excel("OneGas Qualitative R-data-vars_2019-09-18.xlsx")

In [240]:
tbls = dvl.loc[dvl.DataConfigVariable.isnull(),['TableName','ColumnName','DataTableType']].query("DataTableType != 'Virtual' & DataTableType != 'Unknown' & ~DataTableType.isnull()")
tbls

,TableName,ColumnName,DataTableType
0,MOP,DesignPressure,Linear
1,PipeSegment,PipeOutsideDiameter,Linear
2,PipeSegment,PipeWallThickness,Linear
3,PipeSegment,PipeGrade,Linear
15,PipeSegment,PipeToughness,Linear
16,PipeSegment,PipeInserviceDate,Linear
17,PipeSegment,PipeInserviceDate,Linear
18,MAOP,MaxAllowablePressure,Linear
19,PipeSegment,PipeMaterialDomainId,Linear
20,LandUse,LandUseDomainId,Linear


In [244]:
tbls.loc[tbls.ColumnName.str.contains('DomainId'),'DataType'] = 'category'
# tbls.loc[tbls.DataType.isnull(),:]
# ~pd.isnull(tbls.head(1).values[:,3])
tbls

,TableName,ColumnName,DataTableType,DataType
0,MOP,DesignPressure,Linear,NaN
1,PipeSegment,PipeOutsideDiameter,Linear,NaN
2,PipeSegment,PipeWallThickness,Linear,NaN
3,PipeSegment,PipeGrade,Linear,NaN
15,PipeSegment,PipeToughness,Linear,NaN
16,PipeSegment,PipeInserviceDate,Linear,NaN
17,PipeSegment,PipeInserviceDate,Linear,NaN
18,MAOP,MaxAllowablePressure,Linear,NaN
19,PipeSegment,PipeMaterialDomainId,Linear,category
20,LandUse,LandUseDomainId,Linear,category


Declaration of a function that will perform a query to select all data from a table and column given, and close the connection

In [245]:
def get_table_from_IRAS(table,column):
    driver = '{SQL Server Native Client 11.0}'
    server = 'sql2012'
    db = 'og_irasv6_stage'
    conn = pyodbc.connect("Driver="+driver+";Server="+server+";Database="+db+";Trusted_Connection=yes;")

    qd = f"""set nocount on;
            select {column} as [{table}_{column}] from {table}"""
    qdf = pd.read_sql_query(qd,conn)
#     if ~pd.isnull(dtype):
#         qdf = qdf.astype('category')
#     conn.close()
    return qdf.describe(include='all')

APPLY statement is able to perform the query for every combination of table and column given from the filtered dataframe  from the step above

In [247]:
analysis = tbls.apply(lambda x: get_table_from_IRAS(x.TableName,x.ColumnName), axis=1)

Combining all results into a single dataframe for ease of analysis

In [248]:
temp = pd.DataFrame(index=['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'])
for x in analysis:
    temp = pd.merge(temp,x, left_index=True, right_index=True, how='outer')

In [249]:
temp

,MOP_DesignPressure,PipeSegment_PipeOutsideDiameter,PipeSegment_PipeWallThickness,PipeSegment_PipeGrade,PipeSegment_PipeToughness,PipeSegment_PipeInserviceDate_x,PipeSegment_PipeInserviceDate_y,MAOP_MaxAllowablePressure,PipeSegment_PipeMaterialDomainId,LandUse_LandUseDomainId,ClassArea_ClassAreaRatingDomainId,Coating_CoatingLineDomainId,PipeSegment_PipeManufacturerDomainId,PipeSegment_PipeSeamDomainId,OperatingData_OperatingTemperature,PipeSegment_PipeSpecificationDomainId,DamagePrevention_DmgPrevDeviceTypeDomainId,Casing_CasingTypeDomainId,CoatingCondition_CoatingCondLineFoundDomainId,Coating_CoatingInstallDate,Casing_CasingShortedFlg,PipeSegment_PipeGirthWeldDomainId,FrostArea_FrostAreaMin,EarthQuakeArea_EarthQuakeAreaRangeLD,LightningStrikeArea_LightningStrikeAreaMax,TestPostPoint_Id,OperatingData_Id,HCACFRSeg_RCReasonTypeId,HCACFRSeg_Id,InlineInspectionRange_ILIRStartDate,InlineInspectionRange_ILIRStatusDomainId,InlineInspectionFeature_ILIFFeatureNumber,InlineInspectionFeature_ILIFStatusDomainId,InlineInspectionFeature_ILIFSurfaceInd,InlineInspectionFeature_ILIFTypeDomainId,InlineInspectionFeature_ILIFFPR,InlineInspectionNonCrsnDfct_ILINCDFeatureNumber,InlineInspectionNonCrsnDfct_ILINCDTypeDomainId,HydroTestRange_HydroTestRngMinimumPressure,CPSurveyReading_Id,CPSurveyReading_CPSurveyReadingOff,CPSurveyReading_CPSurveyReadingOn,ElevationProfile_InclinationAngle,Fabrication_Id,Fabrication_FabricationTypeDomainId,CoverDepthProfile_CoverDepthReading,Incident_Id,Incident_ThreatTypeDomainId,Incident_DamageCount
25%,3033.693200,4.500000,3.962400,241.316510,NaN,NaN,NaN,1378.951500,7.000000,104.000000,64.000000,113.000000,5.000000,8.000000,21.0,6.000000,NaN,600749.000000,NaN,NaN,0.000000,10.000000,0.254000,600931.0,3.000000,2.75000,63637.750000,17.000000,8303.500000,NaN,100693.000000,NaN,710.0,NaN,94.000000,2.050000,NaN,100709.000000,5171.000000,1.637894e+06,8.720000e+02,1.009000e+03,NaN,3015.250000,NaN,1.143000,1.250000,700.0,NaN
50%,5102.120400,6.625000,5.969000,289.579810,NaN,NaN,NaN,1861.584500,7.000000,106.000000,603681.000000,116.000000,5.000000,600899.000000,21.0,6.000000,NaN,600749.000000,NaN,NaN,0.000000,10.000000,0.508000,600931.0,3.000000,4.50000,63638.500000,17.000000,66276.000000,NaN,100693.000000,NaN,710.0,NaN,100769.000000,2.060000,NaN,100763.000000,7791.000000,2.003994e+06,9.870000e+02,1.184000e+03,NaN,6029.500000,NaN,1.371600,1.500000,700.0,NaN
75%,5102.120400,12.750000,7.112000,358.527380,NaN,NaN,NaN,3240.535900,7.000000,106.000000,603682.000000,600757.000000,5.000000,600899.000000,21.0,56.000000,NaN,600749.000000,NaN,NaN,0.000000,10.000000,0.508000,600931.0,4.000000,6.25000,63639.250000,17.000000,74275.500000,NaN,100693.000000,NaN,710.0,NaN,100769.000000,2.254250,NaN,100770.000000,9928.000000,2.370094e+06,1.111000e+03,1.369000e+03,NaN,9043.750000,NaN,1.752600,1.750000,700.0,NaN
count,7204.000000,21145.000000,20381.000000,20962.000000,0.0,19915,19915,5701.000000,21144.000000,3216.000000,3125.000000,17027.000000,21128.000000,1230.000000,4.0,21141.000000,0.0,973.000000,0.0,10203,394.000000,21144.000000,1435.000000,1422.0,1437.000000,8.00000,4.000000,3553.000000,31999.000000,64,64.000000,12572,12572.0,12572,12572.000000,12572.000000,4610,4610.000000,11271.000000,1.464401e+06,1.464401e+06,1.464401e+06,0.0,12058.000000,0.0,4518.000000,2.000000,2.0,0.0
first,NaN,NaN,NaN,NaN,NaN,1900-01-01 00:00:00,1900-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-09-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,874,874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN,9,NaN,9797,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,2018-11-30 00:00:00,2018-11-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-10-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [250]:
%ls

 Volume in drive Z is Projects
 Volume Serial Number is BC71-F23D

 Directory of Z:\ONEGAS\2019-08_IRAS Implementation\3_Engineering\Results & QC

2019-09-23  01:29 PM    <DIR>          .
2019-09-23  01:29 PM    <DIR>          ..
2019-09-18  02:11 PM    <DIR>          .ipynb_checkpoints
2019-09-10  10:41 AM    <DIR>          Data QC
2019-09-23  01:29 PM           107,833 ONEGAS Data Check.ipynb
2019-09-18  02:04 PM            41,481 OneGas Qualitative R-data-vars_2019-09-18.xlsx
2019-09-18  01:45 PM    <DIR>          Risk Projects
               2 File(s)        149,314 bytes
               5 Dir(s)  81,012,285,440 bytes free
